In [24]:
%matplotlib inline
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import pickle
import h5py
import boost_histogram as bh
import os
import random

In [4]:
positron0 = uproot.open("/mnt/Storage/fmanzali/FlatComplete/eplus_hits_dn_0.root")
electron0 = uproot.open("/mnt/Storage/gvicentini/ElectronDataset/Hits/eminus_hits_dn_0.root")
posPMT0 = positron0["lpmt_hits"]
elPMT0 = electron0['lpmt_hits']

PMT_cor = positron0["lpmt_pos"].pandas.df()

In [5]:
posINFO0 = positron0['true_info'].pandas.df()
elINFO0 = electron0['true_info'].pandas.df()

In [6]:
def no_DN (frame):
    
    i = frame.index[0][0]
    new_frame = frame.loc[(frame['isDN'].values == False) & (frame['hitTime'].values < 300)]
    return new_frame.loc[i]

In [7]:
def first (frame):
    frame_sort = frame.sort_values(['hitTime'])
    frame_first = frame_sort.drop_duplicates(subset='pmtID', keep='first')
    frame_last = frame_first.sort_values('hitTime', ascending=False)
    return frame_last

In [8]:
def normal_ft(ev):
    
    mi = ev['hitTime'].min()
    
    a = ev['nHits'].values
    b = ev['pmtID'].values
    c = ev['hitTime'] - mi
    data = {'nHits': a, 'pmtID': b, 'hitTime': c}
        
    norm = pd.DataFrame(data)
    return norm

In [9]:
def mapev (ev):
    
    x = PMT_cor['pmt_x']
    y = PMT_cor['pmt_y']
    z = PMT_cor['pmt_z']
    
    i = ev['pmtID'].values
    xs = x[ev['pmtID']]
    ys = y[ev['pmtID']]
    zs = z[ev['pmtID']] 
    t = ev['hitTime'].values
    data ={'pmtID': i, 'pmt_x': xs, 'pmt_y': ys, 'pmt_z':zs, 'hitTime': t}
    
    ev_map = pd.DataFrame(data)
    return ev_map

In [10]:
def mollweide (ev_map, j):
    # Set the coordinates for the input
    nside = 32
    npix = hp.nside2npix(nside)

    # Coordinates and the density field f
    hitTime = ev_map['hitTime']
    x = ev_map['pmt_x']
    y = ev_map['pmt_y']
    z = ev_map['pmt_z']
    
    # Go from HEALPix coordinates to indices
    indices = hp.vec2pix(nside, x, y, z)

    # Initate the map and fill it with the values
    hpxmap = np.full(npix, 0, dtype=np.float)
    hpxmap[[indices]]  = 300-hitTime
    cl = hp.anafast(hpxmap)

    # Inspect the map
    #project = hp.mollview(hpxmap, title=None, cbar=False, return_projected_map=True, hold=True) #,cmap='binary')
    #plt.savefig('/mnt/Storage/gvicentini/electron/ev'+str(j)+'.png')  
    #plt.clf()
    return cl #project

In [43]:
def name(value):
    for n,v in globals().items():
        if v == value:
            return n
    return None

In [42]:
def power_spectrum(file, info, n):
    
    if name(file) == 'posPMT'+str(n):
        dst_dir = '/mnt/Storage/gvicentini/positron_spec/'
        data = 'pos_spec'
    elif name(file) == 'elPMT'+str(n):
        dst_dir = '/mnt/Storage/gvicentini/electron_spec/'
        data = 'el_spec'
    else:
        print('file non riconosciuto')
    
    for i in info['evtID']:
        ev = file.pandas.df(entrystart=i, entrystop=i+1)
        try:
            ev_noDN = no_DN(ev)
        except:
            print(i, ': solo dark noise')
            continue
        ev_first = first(ev_noDN)
        ev_norm = normal_ft(ev_first)
        ev_map = mapev(ev_norm)
        cl = mollweide(ev_map, i)
        l = np.arange(len(cl))
        power = l * (l+1) * cl
            
        np.save(dst_dir+str(n)+data+str(i)+'.npy', power)

In [48]:
def time(file, info, n):
    
    if name(file) == 'posPMT'+str(n):
        dst_dir = '/mnt/Storage/gvicentini/positron_time/'
        data = 'pos_time'
    elif name(file) == 'elPMT'+str(n):
        dst_dir = '/mnt/Storage/gvicentini/electron_time/'
        data = 'el_time'
    else:
        print('file non riconosciuto')
    
    hist = bh.Histogram(bh.axis.Regular(20, 0, 300))
    for i in info['evtID']:
        ev = file.pandas.df(entrystart=i, entrystop=i+1)
        try:
            ev_noDN = no_DN(ev)
        except:
            print(i, ': solo dark noise')
            continue
        ev_first = first(ev_noDN)
        ev_norm = normal_ft(ev_first)
        ev_time= ev_norm['hitTime'].values
        hist.fill(ev_time)
        array = hist.to_numpy()
        time = array[0]
        
        np.save(dst_dir+str(n)+data+str(i)+'.npy', time)

In [42]:
power_spectrum(posPMT0, posINFO0, 0)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [14]:
power_spectrum(elPMT0, elINFO0, 0)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


15713 : solo dark noise
17732 : solo dark noise
19836 : solo dark noise
21526 : solo dark noise
21913 : solo dark noise
21979 : solo dark noise
36938 : solo dark noise
48103 : solo dark noise
51121 : solo dark noise
52293 : solo dark noise
60132 : solo dark noise
68127 : solo dark noise
81428 : solo dark noise
94065 : solo dark noise
96584 : solo dark noise


In [49]:
time(posPMT0, posINFO0, 0)

In [50]:
time(elPMT0, elINFO0, 0)

15713 : solo dark noise
17732 : solo dark noise
19836 : solo dark noise
21526 : solo dark noise
21913 : solo dark noise
21979 : solo dark noise
36938 : solo dark noise
48103 : solo dark noise
51121 : solo dark noise
52293 : solo dark noise
60132 : solo dark noise
68127 : solo dark noise
81428 : solo dark noise
94065 : solo dark noise
96584 : solo dark noise
